In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

In [44]:
# Load the dataset
file_path = "data/DeDoDe_Descriptors_Dataset.pth"  # Change this to your actual path
data = torch.load(file_path)
base_descriptors = data['descriptors']  # Assuming these keys exist
transformed_descriptors = data['deformed_descriptors']

/tmp/ipykernel_8174/826197558.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [45]:
from Affine_Transformations import generate_strain_tensors
print(generate_strain_tensors()[31])

(-0.75, 2.0, 0.6)


In [46]:
# Select the data corresponding to transformation 0
transformations = data['transformations']
transformations = torch.repeat_interleave(transformations, repeats=8, dim=1).flatten()
idx = transformations == 31
base_descriptors = base_descriptors[idx]
transformed_descriptors = transformed_descriptors[idx]

In [47]:
print(base_descriptors)
print(transformed_descriptors[0].mean())
print(torch.nn.functional.cosine_similarity(base_descriptors,transformed_descriptors,dim = 1).mean())


tensor([[ 0.8651,  0.2148,  0.1459,  ..., -0.3268,  0.1482,  0.4894],
        [-0.3635,  0.0501,  0.0675,  ..., -0.6345, -0.5220,  0.2216],
        [-0.0612,  0.5054,  0.1808,  ..., -0.2336, -1.0503, -0.0952],
        ...,
        [-0.5589,  0.1584,  0.0729,  ..., -0.6847,  0.3972,  0.2990],
        [ 0.5017,  0.2304, -0.1690,  ...,  0.5677, -0.1402, -0.5549],
        [ 0.9935, -0.2173, -0.8181,  ...,  1.5885,  0.0631, -0.5063]],
       dtype=torch.float64)
tensor(-0.0361, dtype=torch.float64)
tensor(0.0546, dtype=torch.float64)


In [55]:
# Create a dataset and dataloader
dataset = TensorDataset(base_descriptors, transformed_descriptors)
train_size = 0.8
val_size = 0.2

train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [78]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            # nn.Linear(input_dim,output_dim)
            
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
            
            # nn.Linear(input_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.model(x)

In [79]:
# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
input_dim = base_descriptors.shape[1]
output_dim = transformed_descriptors.shape[1]
model = MLP(input_dim,output_dim).double().to(device)

cuda


In [80]:
# Define loss function and optimizer
criterion = nn.CosineEmbeddingLoss()
# criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [81]:
# One eval round before training
cosine_target = torch.ones(4).to(device)

model.eval()
start_loss = 0
with torch.no_grad():
    for base, transformed in val_dataloader:
        base, transformed = base.to(device), transformed.to(device)
        output = model(base)
        # loss = criterion(output,transformed)
        loss = criterion(output, transformed, cosine_target)
        start_loss += loss.item()

print(f'Loss before training: {start_loss/len(val_dataloader)}')

writer = SummaryWriter(log_dir="runs/tain_one_transformation")

# Training loop
epochs = 250
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for base, transformed in train_dataloader:
        base, transformed = base.to(device), transformed.to(device)
        
        optimizer.zero_grad()
        output = model(base)
        # loss = criterion(output, transformed)
        loss = criterion(output, transformed, cosine_target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for base, transformed in val_dataloader:
            base, transformed = base.to(device), transformed.to(device)
            output = model(base)
            # loss = criterion(output,transformed)
            loss = criterion(output, transformed, cosine_target)
            val_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(val_dataloader)

    writer.add_scalar("Loss/Train", avg_train_loss, epoch)
    writer.add_scalar("Loss/Validation", avg_val_loss, epoch)

    if (epoch+1)%10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

Loss before training: 0.9921571320813646
Epoch 10/250, Train Loss: 0.843191, Val Loss: 0.852013
Epoch 20/250, Train Loss: 0.742384, Val Loss: 0.762702
Epoch 30/250, Train Loss: 0.685352, Val Loss: 0.713706
Epoch 40/250, Train Loss: 0.652199, Val Loss: 0.686467
Epoch 50/250, Train Loss: 0.631022, Val Loss: 0.670173
Epoch 60/250, Train Loss: 0.615999, Val Loss: 0.659464
Epoch 70/250, Train Loss: 0.604215, Val Loss: 0.651780
Epoch 80/250, Train Loss: 0.594106, Val Loss: 0.645822
Epoch 90/250, Train Loss: 0.584919, Val Loss: 0.640936
Epoch 100/250, Train Loss: 0.576229, Val Loss: 0.636802
Epoch 110/250, Train Loss: 0.567809, Val Loss: 0.633184
Epoch 120/250, Train Loss: 0.559575, Val Loss: 0.629967
Epoch 130/250, Train Loss: 0.551430, Val Loss: 0.627094
Epoch 140/250, Train Loss: 0.543413, Val Loss: 0.624631
Epoch 150/250, Train Loss: 0.535493, Val Loss: 0.622534
Epoch 160/250, Train Loss: 0.527730, Val Loss: 0.620756
Epoch 170/250, Train Loss: 0.520127, Val Loss: 0.619326
Epoch 180/250, T

In [160]:
# Save the trained model
torch.save(model.state_dict(), "models/single_transformation_model.pth")
print("Model training complete and saved.")

Model training complete and saved.
